In [ ]:
import wandb
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
api = wandb.Api()
all_runs = api.runs('kennychufk/alluvion')

In [ ]:
selected_runs = []
for run in all_runs:
    if (len(run.tags) and run.tags[0]=='vis_sweep2' and run.state=='finished'):
        selected_runs.append(run)

In [ ]:
kinematic_vis = []
vis = []
bvis = []
best_losses = []
for run in selected_runs:
    vis.append(run.summary['vis_real'])
    bvis.append(run.summary['bvis_real'])
#     vis.append(run.summary['best_x'][0])
#     bvis.append(run.summary['best_x'][1])
    kinematic_vis.append(run.config['kinematic_viscosity_real'])
    best_losses.append(run.summary['best_loss'])

In [ ]:
plt.plot(kinematic_vis, vis, marker='x')

In [ ]:
plt.plot(kinematic_vis, bvis, marker='x')

In [ ]:
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",  # use serif/main font for text elements
    "text.usetex": True,     # use inline math for ticks
    "pgf.rcfonts": False,    # don't setup fonts from rc parameters
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 8,
    "font.size": 8,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8
})
plt.style.use('seaborn-whitegrid')
textwidth = 510.295   #pt. # \showthe\textwidth
columnwidth = 243.14749 #pt. # \showthe\columnwidth

def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


In [ ]:
def latex_float(f):
    float_str = "{0:.3g}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str
    
fig, ax = plt.subplots(1, 1, figsize=set_size(columnwidth))

scale = 1e6
vis_color = 'tab:blue'
ax.scatter(np.array(kinematic_vis)*scale, np.array(vis)*scale, marker='o', color=vis_color)
ax.set_ylabel(r"Parameter value (cSt)")

bvis_color = 'tab:red'
ax.scatter(np.array(kinematic_vis)*scale, np.array(bvis)*scale, marker='o', color=bvis_color)

ax.set_xlabel(r"Kinematic viscosity $\nu$ (cSt)")

reg = LinearRegression().fit(np.array(kinematic_vis).reshape(-1, 1), vis)
breg = LinearRegression().fit(np.array(kinematic_vis).reshape(-1, 1), bvis)
xlim_upper_scaled = 22.5
xlim_upper = xlim_upper_scaled * 1e-6
vis_y0 = reg.predict(np.array([0]).reshape(-1, 1))[0]
bvis_y0 = breg.predict(np.array([0]).reshape(-1, 1))[0]
vis_y1 = reg.predict(np.array([xlim_upper]).reshape(-1, 1))[0]
bvis_y1 = breg.predict(np.array([xlim_upper]).reshape(-1, 1))[0]
ax.plot([0, xlim_upper_scaled], np.array([vis_y0, vis_y1])*scale, color=vis_color, label=r"SPH viscosity $\nu_{F}$")
ax.plot([0, xlim_upper_scaled], np.array([bvis_y0, bvis_y1])*scale, color=bvis_color, label=r"SPH boundary viscosity $\nu_{B}$")

vis_coef_str = f"{reg.coef_[0]:.3f}"
vis_intercept_scaled = reg.intercept_*scale
vis_intercept_str = f"{'+' if vis_intercept_scaled>0 else ''}{latex_float(vis_intercept_scaled)}"
ax.annotate(
    r"$"+"\\nu_{B}="+vis_coef_str+"\\nu "+vis_intercept_str+" \\mathrm{cSt} "+r"$",
    xy=(11, 53), xycoords='data',
    bbox=dict(boxstyle="round", facecolor="white", edgecolor=vis_color))

bvis_coef_str = f"{breg.coef_[0]:.3f}"
bvis_intercept_scaled = breg.intercept_*scale
bvis_intercept_str = f"{'+' if bvis_intercept_scaled>0 else ''}{latex_float(bvis_intercept_scaled)}"
ax.annotate(
    r"$"+"\\nu_{B}="+bvis_coef_str+"\\nu "+bvis_intercept_str+" \\mathrm{cSt} "+r"$",
    xy=(1, 85), xycoords='data',
    bbox=dict(boxstyle="round", facecolor="white", edgecolor=bvis_color))


ax.set_xlim(0,xlim_upper_scaled)
ax.set_ylim(0,150)
ax.legend(facecolor='white', frameon=True, edgecolor='None')
fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633
fig.savefig('vary-vis.pgf')

In [ ]:
plt.plot(kinematic_vis, best_losses)

In [ ]:
reg.coef_[0], reg.intercept_

In [ ]:
breg.coef_[0], breg.intercept_